In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import ResNet50_Weights # Импортируем ResNet50_Weights
import os

# 1. Определение преобразований для изображений
# Нормализация данных. Значения, основанные на ImageNet, подходят для предобученных моделей.
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



In [7]:
# 2. Загрузка данных
data_dir = r'C:\trainData\split' # Укажите путь к вашему датасету
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32,
                             shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 3. Использование предобученной модели (Transfer Learning)
# ResNet-18 - отличный выбор для начала
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Заморозка всех слоев, кроме последнего, для сохранения обученных весов
for param in model.parameters():
    param.requires_grad = False

# Замена последнего полносвязного слоя на новый
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # У нас 2 класса: 'pathology' и 'no_pathology'

model = model.to(device)

if torch.cuda.is_available():
    print("GPU доступен. Используется:", torch.cuda.get_device_name(0))
    print("Количество GPU:", torch.cuda.device_count())
    device = torch.device("cuda:0")
else:
    print("GPU не найден. Обучение будет на CPU.")
    device = torch.device("cpu")

print(torch.version.cuda)

GPU доступен. Используется: NVIDIA GeForce RTX 2070
Количество GPU: 1
12.1


In [ ]:
import copy
# 4. Определение функции потерь, оптимизатора и планировщика
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
# Планировщик будет уменьшать lr в 10 раз каждые 7 эпох
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# 5. Функция обучения (с Early Stopping)
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    # Параметры Early Stopping
    epochs_no_improve = 0
    n_epochs_stop = 50
    
    for epoch in range(num_epochs):
        print(f'Эпоха {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            # Обновление планировщика только на этапе обучения
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Потери: {epoch_loss:.4f} Точность: {epoch_acc:.4f}')

            # Ранняя остановка и сохранение лучшей модели
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                
                if epochs_no_improve >= n_epochs_stop:
                    print('Ранняя остановка! Обучение завершено.')
                    model.load_state_dict(best_model_wts)
                    torch.save(model.state_dict(), 'best_model_weights.pt')
                    return model

    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_model_weights.pt')
    return model

# 6. Запуск обучения
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=50)

Эпоха 0/49
----------
train Потери: 0.3324 Точность: 0.8739
val Потери: 0.2673 Точность: 0.9120
Эпоха 1/49
----------
train Потери: 0.2483 Точность: 0.9077
val Потери: 0.2423 Точность: 0.9195
Эпоха 2/49
----------
train Потери: 0.2245 Точность: 0.9168
val Потери: 0.2133 Точность: 0.9204
Эпоха 3/49
----------
train Потери: 0.2134 Точность: 0.9240
val Потери: 0.2065 Точность: 0.9190
Эпоха 4/49
----------
train Потери: 0.1992 Точность: 0.9302
val Потери: 0.1996 Точность: 0.9260
Эпоха 5/49
----------
train Потери: 0.1900 Точность: 0.9327
val Потери: 0.2221 Точность: 0.9330
Эпоха 6/49
----------
train Потери: 0.1854 Точность: 0.9334
val Потери: 0.1927 Точность: 0.9309
Эпоха 7/49
----------
train Потери: 0.1721 Точность: 0.9379
val Потери: 0.1865 Точность: 0.9291
Эпоха 8/49
----------
train Потери: 0.1752 Точность: 0.9389
val Потери: 0.1885 Точность: 0.9330
Эпоха 9/49
----------
train Потери: 0.1761 Точность: 0.9367
val Потери: 0.1798 Точность: 0.9313
Эпоха 10/49
----------
train Потери: 0.1

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import ResNet50_Weights
import os
import copy

# 1. Определение преобразований с улучшенной аугментацией
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15), 
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        # --- Добавленные методы аугментации ---
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Изменение цвета
        transforms.RandomPerspective(distortion_scale=0.3, p=0.5), # Изменение перспективы
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 2. Загрузка данных
data_dir = r'C:\trainData\split'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32,
                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# 3. Настройка GPU/CPU
if torch.cuda.is_available():
    print("GPU доступен. Используется:", torch.cuda.get_device_name(0))
    print("Количество GPU:", torch.cuda.device_count())
    device = torch.device("cuda:0")
else:
    print("GPU не найден. Обучение будет на CPU.")
    device = torch.device("cpu")
print(f"Версия CUDA: {torch.version.cuda}")

# 4. Использование предобученной модели (ResNet-50)
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

# 5. Определение функции потерь, оптимизатора и планировщика
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# 6. Функция обучения (с Early Stopping)
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    epochs_no_improve = 0
    n_epochs_stop = 10
    
    for epoch in range(num_epochs):
        print(f'Эпоха {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Потери: {epoch_loss:.4f} Точность: {epoch_acc:.4f}')

            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                
                if epochs_no_improve >= n_epochs_stop:
                    print('Ранняя остановка! Обучение завершено.')
                    model.load_state_dict(best_model_wts)
                    torch.save(model.state_dict(), 'best_model_weights.pt')
                    return model

    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_model_weights.pt')
    return model

# 7. Запуск обучения
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=70)

GPU доступен. Используется: NVIDIA GeForce RTX 2070
Количество GPU: 1
Версия CUDA: 12.1
Эпоха 0/69
----------
train Потери: 0.1992 Точность: 0.9269
val Потери: 0.1420 Точность: 0.9497
Эпоха 1/69
----------
train Потери: 0.1305 Точность: 0.9574
val Потери: 0.1242 Точность: 0.9615
Эпоха 2/69
----------
train Потери: 0.1107 Точность: 0.9648
val Потери: 0.1424 Точность: 0.9606
Эпоха 3/69
----------
train Потери: 0.1007 Точность: 0.9674
val Потери: 0.1003 Точность: 0.9650
Эпоха 4/69
----------
train Потери: 0.0902 Точность: 0.9709
val Потери: 0.0925 Точность: 0.9711
Эпоха 5/69
----------
train Потери: 0.0874 Точность: 0.9708
val Потери: 0.0913 Точность: 0.9689
Эпоха 6/69
----------
train Потери: 0.0838 Точность: 0.9720
val Потери: 0.0732 Точность: 0.9742
Эпоха 7/69
----------
train Потери: 0.0592 Точность: 0.9805
val Потери: 0.0723 Точность: 0.9768
Эпоха 8/69
----------
train Потери: 0.0567 Точность: 0.9815
val Потери: 0.0770 Точность: 0.9781
Эпоха 9/69
----------
train Потери: 0.0466 Точно

In [38]:
torch.save(model.state_dict(), r"C:\trainData\split\VAR3.pth")

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import ResNet18_Weights

# Инициализация той же архитектуры модели
model_path = r"C:\trainData\split\VAR3.pth"  # Путь к вашему файлу с весами
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Заменяем последний слой, как и при обучении
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Загружаем сохранённые веса
model.load_state_dict(torch.load(model_path))
model.eval()  # Переводим модель в режим оценки

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Преобразования для инференса
inference_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Путь к папке с изображениями для предсказания
inference_dir = r"C:\testData\norma_anon\postDICOM"
inference_dataset = datasets.ImageFolder(inference_dir, inference_transforms)
inference_loader = DataLoader(inference_dataset, batch_size=32, shuffle=False)

# Убедитесь, что классы соответствуют тем, на которых обучалась модель


In [40]:

from PIL import Image
# 2. Подготовка преобразований для одного изображения (без изменений)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 3. Путь к папке с изображениями для предсказания (без изменений)
image_folder_path = r"C:\testData\pneumonia_anon\ostDICOM_black"

# Имена классов, в том же порядке, что и при обучении
class_names = [ 'pathology', 'no_pathology']

pathology_count = 0
no_pathology_count = 0
pathology_confidence_sum = 0.0
no_pathology_confidence_sum = 0.0
total_images = 0

# 4. Проход по каждому изображению и получение предсказания
with torch.no_grad():
    for filename in os.listdir(image_folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder_path, filename)
            
            try:
                image = Image.open(image_path).convert('RGB')
                image = transform(image).unsqueeze(0).to(device)
                outputs = model(image)
                
                probabilities = torch.nn.functional.softmax(outputs, dim=1)[0]
                
                confidence, predicted_class_index = torch.max(probabilities, 0)
                predicted_class_name = class_names[predicted_class_index.item()]
                confidence_score = confidence.item()

                print(f"Изображение: {filename}, Предсказанный класс: {predicted_class_name}, Уверенность: {confidence_score * 100:.2f}%")

                # Увеличение счетчиков и сумм уверенности
                if predicted_class_name == 'pathology':
                    pathology_count += 1
                    pathology_confidence_sum += confidence_score
                else:
                    no_pathology_count += 1
                    no_pathology_confidence_sum += confidence_score
                total_images += 1
            
            except Exception as e:
                print(f"Не удалось обработать файл {filename}. Ошибка: {e}")

# 5. Вывод итоговой статистики
print("\n--- Результаты классификации ---")
print(f"Всего обработано изображений: {total_images}")
print(f"Изображений с патологией: {pathology_count}")
print(f"Изображений без патологии: {no_pathology_count}")

# Расчет и вывод средней уверенности
if pathology_count > 0:
    avg_pathology_confidence = pathology_confidence_sum / pathology_count
    print(f"Средняя уверенность для класса 'pathology': {avg_pathology_confidence * 100:.2f}%")
else:
    print("Изображения с патологией не найдены.")

if no_pathology_count > 0:
    avg_no_pathology_confidence = no_pathology_confidence_sum / no_pathology_count
    print(f"Средняя уверенность для класса 'no_pathology': {avg_no_pathology_confidence * 100:.2f}%")
else:
    print("Изображения без патологии не найдены.")

Изображение: 1000032A_anon.png, Предсказанный класс: pathology, Уверенность: 88.60%
Изображение: 1000032B_anon.png, Предсказанный класс: pathology, Уверенность: 78.82%
Изображение: 1000032C_anon.png, Предсказанный класс: pathology, Уверенность: 78.35%
Изображение: 1000032D_anon.png, Предсказанный класс: pathology, Уверенность: 59.34%
Изображение: 1000032E_anon.png, Предсказанный класс: pathology, Уверенность: 86.40%
Изображение: 1000032F_anon.png, Предсказанный класс: pathology, Уверенность: 93.38%
Изображение: 10000330_anon.png, Предсказанный класс: pathology, Уверенность: 97.70%
Изображение: 10000331_anon.png, Предсказанный класс: pathology, Уверенность: 94.21%
Изображение: 10000332_anon.png, Предсказанный класс: pathology, Уверенность: 90.78%
Изображение: 10000333_anon.png, Предсказанный класс: pathology, Уверенность: 90.91%
Изображение: 10000334_anon.png, Предсказанный класс: pathology, Уверенность: 79.20%
Изображение: 10000335_anon.png, Предсказанный класс: no_pathology, Увереннос